# Neural search for question answering
The exercise introduces the problem of factual question answering. This part concentrates on the methods for retrieving the content of documents that potentialy might be useful for answering the question. We compare sparse text representations (e.g. ElasticSearch default behaviour), with dense text representations (e.g. Dense Passage Retriever bi-encoder neural model).

## Install Haystack framework. You may need to used this fork to get support for Polish QA models.

In [ ]:
!pip install transformers
!pip install git+https://github.com/apohllo/haystack.git@b79c5b099b294ad5f6cf37a01e4c504b438b8018

     |████████████████████████████████| 3.4 MB 5.4 MB/s 
     |████████████████████████████████| 596 kB 47.8 MB/s 
     |████████████████████████████████| 895 kB 45.0 MB/s 
     |████████████████████████████████| 3.3 MB 36.6 MB/s 
     |████████████████████████████████| 61 kB 539 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Cloning https://github.com/apohllo/haystack.git (to revision b79c5b099b294ad5f6cf37a01e4c504b438b8018) to /tmp/pip-req-build-micgwy47
  Running command git clone -q https://github.com/apohllo/haystack.git /tmp/pip-req-build-micgwy47
  Running command git rev-parse -q --verify 'sha^b79c5b099b294ad5f6cf37a01e4c504b438b8018'
  Running command git fetch -q https://github.com/apohllo/haystack.git b79c5b099b294ad5f6cf37a01e4c504b438b8018
  Running command git checkout -q b79c5b099b294ad5f6cf37a01e4c504b438b8018
     |████████████████████████████████| 43 kB 113 k

## Start Elasticsearch

In [1]:
from haystack.document_stores import FAISSDocumentStore, ElasticsearchDocumentStore
from haystack.nodes import DensePassageRetriever
from haystack.nodes import ElasticsearchRetriever
from tqdm import tqdm
import sys
import os
import regex
import random
import csv
import time
import pandas as pd

/usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)


In [31]:
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  
                  )
# wait until ES has started
! sleep 30

In [27]:
!curl -X GET "localhost:9200/"

{
  "name" : "c32337b4f939",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "1tDmfih3TvasmYfSRNeA_A",
  "version" : {
    "number" : "7.9.2",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "d34da0ea4a966c4e49417f2da2f244e3e97b4e6e",
    "build_date" : "2020-09-23T00:45:33.626720Z",
    "build_snapshot" : false,
    "lucene_version" : "8.6.2",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [ ]:
# es_server.kill()

## Configure one document store based on ElasticSearch and another document store based on Faiss supported by DPR

In [5]:
elastic_store = ElasticsearchDocumentStore()

In [20]:
faiss_store = FAISSDocumentStore(faiss_index_factory_str="Flat")

In [7]:
faiss_store = FAISSDocumentStore.load(index_path="drive/MyDrive/acts.faiss")

In [6]:
elastic_retriever = ElasticsearchRetriever(elastic_store)

In [21]:
faiss_retriever = DensePassageRetriever(
    document_store=faiss_store,
    query_embedding_model="enelpol/czywiesz-question",
    passage_embedding_model="enelpol/czywiesz-context",
)

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1


Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/472 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/886k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/543k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/543k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/129 [00:00<?, ?B/s]

INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find enelpol/czywiesz-context locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...


Downloading:   0%|          | 0.00/475M [00:00<?, ?B/s]

INFO - haystack.modeling.model.language_model -  Loaded enelpol/czywiesz-context
WARNING - haystack.modeling.logger -  Failed to log params: Changing param values is not allowed. Param with key='lm1_name' was already logged with value='facebook/dpr-question_encoder-single-nq-base' for run ID='6f092cd2ac794094b2c3409ff69ae00b'. Attempted logging new value 'enelpol/czywiesz-question'.


In [9]:
faiss_retriever = DensePassageRetriever.load("drive/MyDrive/retriever.pt", document_store=faiss_store)

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Model found locally at drive/MyDrive/retriever.pt/query_encoder
INFO - haystack.modeling.model.language_model -  Loaded drive/MyDrive/retriever.pt/query_encoder
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Model found locally at drive/MyDrive/retriever.pt/passage_encoder
INFO - haystack.modeling.model.language_model -  Loaded drive/MyDrive/retriever.pt/passage_encoder
WARNING - haystack.modeling.logger -  Failed to log params: Changing param values is not allowed. Param with key='lm1_name' was already logged with value='facebook/dpr-question_encoder-single-nq-base' for run ID='6f0

## Pre-process all documents from the set of Polish bills (used in the previous exercises), but splitting them into individual articles

In [10]:
ustawy_path = 'drive/MyDrive/ustawy'
legislation_acts = {}

with tqdm(total=1179, file=sys.stdout) as pbar: 
    for filename in os.listdir(ustawy_path):
            with open(ustawy_path + '/' + filename, 'r', encoding='utf8') as f:
                content = f.read()
                legislation_acts[filename] = content
                pbar.update(1)

100%|██████████| 1179/1179 [00:16<00:00, 72.99it/s] 


In [11]:
newline_regex = regex.compile(r'\s+')

def retrieve_articles_from(act):
    result = []
    expected_art_number = 1
    prev_start_pos = None
    for art in regex.compile(r"Art\.\s(\d+)").finditer(act):
        art_number = art.group(1)
        if art_number == str(expected_art_number):
            start_pos = art.start(0)
            if prev_start_pos is not None:
              result.append((expected_art_number - 1, newline_regex.sub(' ', act[prev_start_pos:start_pos])))
            expected_art_number += 1
            prev_start_pos = start_pos
    if prev_start_pos is not None:
      result.append((expected_art_number - 1, newline_regex.sub(' ', act[prev_start_pos:start_pos])))
         
    return result

articles = []
for name, act in tqdm(legislation_acts.items()):
  for art, content in retrieve_articles_from(act):
    articles.append({
        "content": content,
        "meta": {
         "act_name": name,
         "article": art
        }
    })

100%|██████████| 1179/1179 [00:02<00:00, 540.42it/s]


## Load the passages from previous point to the document stores described in point 2.

In [12]:
elastic_store.write_documents(articles)

In [22]:
faiss_store.write_documents(articles)

Writing Documents:   0%|          | 0/22308 [00:00<?, ?it/s]

In [24]:
faiss_store.update_embeddings(faiss_retriever)

INFO - haystack.document_stores.faiss -  Updating embeddings for 21126 docs...


Updating Embedding:   0%|          | 0/21126 [00:00<?, ? docs/s]

/usr/local/lib/python3.7/dist-packages/haystack/modeling/data_handler/dataset.py:65: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  cur_tensor = torch.tensor([sample[t_name] for sample in features], dtype=torch.long)


Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/1136 [00:00<?, ? Docs/s]

In [ ]:
faiss_store.save('drive/MyDrive/acts.faiss')

In [ ]:
faiss_retriever.save("drive/MyDrive/retriever.pt")

INFO - haystack.modeling.model.biadaptive_model -  prediction_head saving


## Invent 30 factual questions based directly on 30 distinct passages that you have selected. The larger number of randomly selected passages should allow to skip those that are hard for inventing a question for them.

In [13]:
art = random.choice(articles)
print(art['content'])
print(art['meta'])

Art. 31. 1. Pracodawca jest obowiązany regularnie kontrolować okresy prowadzenia pojazdu, czas trwania innej pracy oraz okresy odpoczynku, na podstawie zapisów przyrządu kontrolnego, a w przypadku pojazdów nie wyposażonych w te przyrządy, na podstawie karty drogowej. 2. Zapisy z przyrządów kontrolnych pracodawca przechowuje przez okres co najmniej dwunastu miesięcy po wykorzystaniu i udostępnia ich kopie na wniosek kierowców, których one dotyczą. Zapisy te są okazywane lub wydawane na żądanie każdej osoby uprawnionej do przeprowadzenia kontroli. 
{'act_name': '2001_1354.txt', 'article': 31}


In [14]:
re = regex.compile(",|;")
articles_dict = {(art['meta']['act_name'], art['meta']['article']): art['content'] for art in articles}

class Question:
  def __init__(self, question, act_name, article_number):
    self.question = question
    self.passage_id = f"{act_name}/Art.{article_number}" 
    self.passage = re.sub(" ", articles_dict[(act_name, article_number)])

  def __str__(self):
    return f"({self.question} | {self.passage_id} | {self.passage})"

  def get_row(self):
    return (self.passage_id, self.question, self.passage)  

In [15]:
questions = [
  Question("Gdzie agent morski podejmuje się za wynagrodzeniem stałego przedstawicielstwa armatora?", "2001_1545.txt", 201),
  Question("Kiedy zostaną powołani sędziowie łowieccy oraz rzecznicy dyscyplinarni i ich zastępcy pierwszej kadencji?", "2003_844.txt", 5),
  Question("Kto określi wysokość składki na ubezpiecznie społeczne?", "1997_777.txt", 10),
  Question("Od czego rozpoczyna się przewód sądowy?", "1997_735.txt", 72),
  Question("Jakiej karze podlega osoba wprowadzająca do obrotu produkt kosmetyczny bez spełnienia wymogów dotyczących dokumentacji produktu?", "2004_623.txt", 30),
  Question("Czy wydatki budżetowe ujęte w budżecie państwa w części obrona narodowa mogą być wykorzystane na inne cele?", "1997_686.txt", 19),
  Question("Czy zamknięcie składowiska odpadów lub jego wydzielonej części wymaga zgody właściwego organu?", "2001_628.txt", 54),
  Question("Kto korzysta z ochrony przewidzianej w Kodeksie karnym dla funkcjonariuszy publicznych?", "1996_460.txt", 95),
  Question("Gdzie powinna być prowadzona działalność gospodarcza w zakresie wytwarzania środków żywienia zwierząt?", "1997_783.txt", 5),
  Question("Z kim współdziała dyrektor zakładu karnego dla realizacji celów kary?", "1997_557.txt", 236),
  Question("Zgodnie z czym izby prowadzą rachunkowość i sprawozdawczość?", "1996_3.txt", 36),
  Question("Kto ustali termin, w którym podejmą działalność izby Sądu?", "1995_368.txt", 65),
  Question("Kto odpowiada za ochronę informacji niejawnych?", "1999_95.txt", 18),
  Question("Do czego emitowania w publicznych środkach masowego przekazu ma prawo Kościół?", "1997_553.txt", 11),
  Question("Ile powinien wynosić co najmniej kapitał zakładowy spółki?", "1996_465.txt", 154),
  Question("Z jakim dniem tworzy się urzędy marszałkowskie w miastach będących siedzibami władz samorządu województwa?", "1994_592.txt", 27),
  Question("Od czego uzależnione jest korzystanie z gospodarczej procedury celnej?", "1997_117.txt", 90),
  Question("Kto orzeka o pozbawieniu, obniżeniu lub utracie stopnia?", "2001_298.txt", 49),
  Question("Jakiemu żołnierzowi przysługuje renta inwalidzka?", "1997_501.txt", 19),
  Question("Co mogą powołać podmioty gospodarcze prowadzące działalność na terenie strefy?", "1994_600.txt", 11),
  Question("Z czego wyłącznie mogą być produkowane wina gronowe?", "2001_1401.txt", 26),
  Question("Co tworzy Skarb Państwa w celu przygotowania i wykonania przedsięwzięć Euro 2012?", "2004_42.txt", 7),
  Question("Co należy do obowiązków osoby wykwalifikowanej odpowiedzialnej za prowadzenie hurtowni farmaceutycznej?", "2001_1381.txt", 85),
  Question("Z kim współpracują organy kontroli, rewizji, inspekcji działające w administracji rządowej i samorządzie terytorialnym?", "1995_59.txt", 12),
  Question("W jaki sposób odpowiada sędzia za wykroczenia?", "2001_1070.txt", 81),
  Question("Czyim zadaniem jest ochrona przed powodzią oraz suszą?", "2001_1229.txt", 81),
  Question("W jakim wymiarze żołnierze zawodowi otrzymują corocznie urlop wypoczynkowy?", "2003_1750.txt", 61),
  Question("Czego nie powinny zawierać materiały i wyroby przeznaczone do kontaktu z żywnością?", "2001_634.txt", 26),
  Question("Jakie zwierzęta jako dobro ogólnonarodowe stanowią własność Skarbu Państwa?", "1995_713.txt", 2),
  Question("Kiedy wygasają wzajemne zobowiązania stron?", "2001_1545.txt", 163)
]

In [16]:
assert(len(questions) == 30)

In [17]:
file_name = "mikolaj_sikora_questions.csv"

In [29]:
with open(file_name, "w", encoding="utf-8") as file:
  writer = csv.writer(file, delimiter=";")
  writer.writerow(("passage_id", "question", "passage"))
  for question in questions:
    writer.writerow(question.get_row())

## Use the set of questions defined in the previous point to assess the performance of the document stores

In [32]:


with open(file_name, "a") as f:
    writer = csv.writer(f, delimiter=';')
    for question in tqdm(questions):
        print(question.question, question.passage_id)
        for retriever in [elastic_retriever, faiss_retriever]:
            if retriever == elastic_retriever:
              print("Elastic")
            else:
              print("FAISS")  

            possible = retriever.retrieve(query=question.question,top_k=3)

            for q in possible:
                print(q.content)
                print(q.meta["act_name"])
                if input() == "y":
                    writer.writerow(Question(question.question, q.meta["act_name"], int(q.meta["article"])).get_row())
                    f.flush()

  0%|          | 0/30 [00:00<?, ?it/s]

Gdzie agent morski podejmuje się za wynagrodzeniem stałego przedstawicielstwa armatora? 2001_1545.txt/Art.201
Elastic
Art. 201. Przez umowę agencyjną, agent morski podejmuje się za wynagrodzeniem stałego przedstawicielstwa armatora w określonym porcie lub na określonym obszarze. 
2001_1545.txt
y
Art. 202. § 1. Agent morski jest uprawniony do podejmowania w imieniu armatora zwykłych czynności związanych z uprawianiem żeglugi. § 2. W szczególności agent morski jest uprawniony do działania w imieniu armatora wobec urzędów i podmiotów zarządzających portami morskimi, do załatwiania dla statku wszelkich czynności i przyjmowania oświadczeń związanych z przyjściem, postojem i wyjściem statku, do zawierania w imieniu armatora umów przewozu, umów ubezpieczenia morskiego oraz umów przeładunku i składu, do wystawiania konosamentów, do odbioru i zapłaty wszelkich należności związanych z zawinięciem statku do portu i przewozem ładunku lub pasażerów oraz do dochodzenia w imieniu armatora roszczeń wy

  3%|▎         | 1/30 [00:08<04:12,  8.70s/it]

Kiedy zostaną powołani sędziowie łowieccy oraz rzecznicy dyscyplinarni i ich zastępcy pierwszej kadencji? 2003_844.txt/Art.5
Elastic
Art. 5. 1. Sędziowie łowieccy oraz rzecznicy dyscyplinarni i ich zastępcy pierwszej kadencji zostaną powołani w terminie 90 dni od dnia wejścia w życie niniejszej ustawy. 2. Sprawy dyscyplinarne w toku zostaną przekazane rzecznikom dyscyplinarnym, powołanym zgodnie z art. 35o ust. 1 pkt 2 i ust. 2 pkt 2 ustawy zmienianej w art. 1, w brzmieniu nadanym niniejsza ustawą, w terminie 30 dni od dnia ich powołania. 
2003_844.txt
y
Art. 49. Członkowie Rady pierwszej kadencji zostaną powołani w terminie 3 miesięcy od dnia wejścia w życie niniejszej ustawy. 
2003_1834.txt

Art. 80. Z dniem wejścia w życie ustawy członkowie Rady Agencji Własności Rolnej Skarbu Państwa, powołani spośród kandydatów przedstawianych przez organizacje społeczne i zawodowe organizacji związkowych rolników, rolnicze związki spółdzielcze oraz organizacje pracodawców rolnych, stają się człon

  7%|▋         | 2/30 [00:17<04:07,  8.83s/it]

...

 93%|█████████▎| 28/30 [05:48<00:19,  9.57s/it]

Jakie zwierzęta jako dobro ogólnonarodowe stanowią własność Skarbu Państwa? 1995_713.txt/Art.2
Elastic
Art. 2. Zwierzęta łowne w stanie wolnym, jako dobro ogólnonarodowe, stanowią własność Skarbu Państwa. 
1995_713.txt
y
Art. 21. Zwierzęta wolno żyjące stanowią dobro ogólnonarodowe i powinny mieć zapewnione warunki rozwoju i swobodnego bytu, z wyjątkiem tych, o których mowa w art. 23 ust. 1. 
1997_724.txt

Art. 10. 1. Wody stanowią własność Skarbu Państwa, innych osób prawnych albo osób fizycznych. 2. Wody stanowiące własność Skarbu Państwa lub jednostek samorządu terytorialnego są wodami publicznymi. 3. Płynące wody publiczne nie podlegają obrotowi cywilnoprawnemu, z wyjątkiem przypadków określonych w ustawie. 
2001_1229.txt

FAISS
Art. 2. Zwierzęta łowne w stanie wolnym, jako dobro ogólnonarodowe, stanowią własność Skarbu Państwa. 
1995_713.txt

Art. 3. 1. Ministrowi Skarbu Państwa przysługują, określone w odrębnych przepisach, uprawnienia względem państwowych osób prawnych w rozumie

 97%|█████████▋| 29/30 [05:53<00:08,  8.09s/it]

Kiedy wygasają wzajemne zobowiązania stron? 2001_1545.txt/Art.163
Elastic
Art. 163. § 1. Wzajemne zobowiązania stron wygasają, jeżeli po zawarciu umowy, a przed odejściem statku z miejsca załadowania z przyczyn, za które żadna ze stron nie odpowiada: 1) statek oznaczony w umowie zaginął, został zabrany jako łup albo uległ wypadkowi, wskutek którego stał się niezdatny do naprawy lub jej niewart, albo 2) ładunek indywidualnie oznaczony w umowie zaginął. § 2. Jeżeli okoliczności przewidziane w § 1 nastąpiły w czasie podróży, przewoźnik zachowuje prawo do frachtu dystansowego, gdy wypadkowi uległ statek, a cały ładunek ocalał, prawo zaś do części frachtu dystansowego przypadającej od ocalonej części ładunku, gdy wypadkowi uległ ładunek. 
2001_1545.txt
y
Art. 2. 1. Każdej ze stron Komisji przysługuje prawo wniesienia pod obrady Komisji sprawy o dużym znaczeniu społecznym lub gospodarczym, jeżeli uzna, że jej rozwiązanie jest istotne dla zachowania pokoju społecznego. 2. Każda ze stron Komis

100%|██████████| 30/30 [05:59<00:00, 11.98s/it]


In [33]:
final_file_name = "mikolaj_sikora_final_questions1.csv"


In [34]:
with open(file_name, "r") as duplicated:
  with open(final_file_name, "w") as final:
    reader = csv.reader(duplicated, delimiter=";")
    writer = csv.writer(final, delimiter=";")
    keys = set([])
    for r in reader:
      key = r[0]
      if key not in keys:
        writer.writerow(r)
        keys.add(key)

In [35]:
answers = {}
with open(final_file_name, 'r') as f:
    reader = csv.reader(f, delimiter=';')
    for i, r in enumerate(reader):
      if i > 0:
        if r[1] in answers:
            answers[r[1]].append(r[0])
        else:
            answers[r[1]] = [r[0]]

In [36]:
answers

{'Co mogą powołać podmioty gospodarcze prowadzące działalność na terenie strefy?': ['1994_600.txt/Art.11'],
 'Co należy do obowiązków osoby wykwalifikowanej odpowiedzialnej za prowadzenie hurtowni farmaceutycznej?': ['2001_1381.txt/Art.85'],
 'Co tworzy Skarb Państwa w celu przygotowania i wykonania przedsięwzięć Euro 2012?': ['2004_42.txt/Art.7',
  '2004_42.txt/Art.13',
  '2004_42.txt/Art.18',
  '2004_42.txt/Art.2',
  '2004_42.txt/Art.26'],
 'Czego nie powinny zawierać materiały i wyroby przeznaczone do kontaktu z żywnością?': ['2001_634.txt/Art.26'],
 'Czy wydatki budżetowe ujęte w budżecie państwa w części obrona narodowa mogą być wykorzystane na inne cele?': ['1997_686.txt/Art.19',
  '2000_85.txt/Art.34'],
 'Czy zamknięcie składowiska odpadów lub jego wydzielonej części wymaga zgody właściwego organu?': ['2001_628.txt/Art.54'],
 'Czyim zadaniem jest ochrona przed powodzią oraz suszą?': ['2001_1229.txt/Art.81'],
 'Do czego emitowania w publicznych środkach masowego przekazu ma prawo

## Compare the performance of the data stores using the following metrics: Pr@1, Rc@1, Pr@3, Rc@3.

In [37]:
def processQuestion(question, retriever, n):
    start = time.time()
    possible = retriever.retrieve(query=question.question, top_k=n)
    i = 0
    for answer in possible:
        act_name = answer.meta["act_name"]
        article_number = answer.meta["article"]
        if f"{act_name}/Art.{article_number}" in answers[question.question]:
            i += 1
    end = time.time()
    return (1.0*i/n, 1.0*i/(i+n), end - start)

def get_results(n):
    results = []
    for question in questions:
        elastic_pr, elastic_rc, elastic_time = processQuestion(question, elastic_retriever, n)
        faiss_pr, faiss_rc, faiss_time = processQuestion(question, faiss_retriever, n)
        results.append((elastic_pr, elastic_rc, elastic_time, faiss_pr, faiss_rc, faiss_time))
    return pd.DataFrame(data=results, columns=[f"Pr@{n}_elastic", f"Rc@{n}_elastic", "time_elastic", f"Pr@{n}_faiss", f"Rc@{n}_faiss", "time_faiss"])



In [38]:
results1 = get_results(1)
results1

,Pr@1_elastic,Rc@1_elastic,time_elastic,Pr@1_faiss,Rc@1_faiss,time_faiss
0,1.0,0.5,0.024994,1.0,0.5,0.054587
1,1.0,0.5,0.012917,1.0,0.5,0.055476
2,1.0,0.5,0.009332,0.0,0.0,0.049022
3,1.0,0.5,0.012557,0.0,0.0,0.057370
4,1.0,0.5,0.019245,1.0,0.5,0.059224
5,1.0,0.5,0.018619,1.0,0.5,0.054980
6,1.0,0.5,0.017437,0.0,0.0,0.080364
7,1.0,0.5,0.020430,1.0,0.5,0.064448
8,1.0,0.5,0.015665,1.0,0.5,0.056992
9,1.0,0.5,0.012187,0.0,0.0,0.056548


In [39]:
results3 = get_results(3)
results3

,Pr@3_elastic,Rc@3_elastic,time_elastic,Pr@3_faiss,Rc@3_faiss,time_faiss
0,0.333333,0.25,0.019665,0.333333,0.25,0.058279
1,0.333333,0.25,0.021247,0.333333,0.25,0.056019
2,0.666667,0.40,0.015403,0.000000,0.00,0.056913
3,1.000000,0.50,0.012685,0.333333,0.25,0.056615
4,1.000000,0.50,0.019468,1.000000,0.50,0.056306
5,0.666667,0.40,0.011004,0.666667,0.40,0.049547
6,0.333333,0.25,0.014283,0.000000,0.00,0.051966
7,1.000000,0.50,0.011824,1.000000,0.50,0.054590
8,0.333333,0.25,0.012673,0.333333,0.25,0.065249
9,0.333333,0.25,0.008339,0.000000,0.00,0.049604


In [40]:
print("Mean Pr@1")
print(f"Elastic: {results1['Pr@1_elastic'].mean()}")
print(f"Faiss: {results1['Pr@1_faiss'].mean()}")

Mean Pr@1
Elastic: 1.0
Faiss: 0.5333333333333333


In [41]:
print("Mean Rc@1")
print(f"Elastic: {results1['Rc@1_elastic'].mean()}")
print(f"Faiss: {results1['Rc@1_faiss'].mean()}")

Mean Rc@1
Elastic: 0.5
Faiss: 0.26666666666666666


In [42]:
print("Mean Pr@3")
print(f"Elastic: {results3['Pr@3_elastic'].mean()}")
print(f"Faiss: {results3['Pr@3_faiss'].mean()}")

Mean Pr@3
Elastic: 0.5333333333333333
Faiss: 0.3444444444444445


In [43]:
print("Mean Rc@3")
print(f"Elastic: {results3['Rc@3_elastic'].mean()}")
print(f"Faiss: {results3['Rc@3_faiss'].mean()}")

Mean Rc@3
Elastic: 0.33000000000000007
Faiss: 0.22000000000000003


In [44]:
print("Mean times")
print(f"Elastic: {results1['time_elastic'].mean()}")
print(f"Faiss: {results1['time_faiss'].mean()}")

Mean times
Elastic: 0.013444455464680989
Faiss: 0.054728428522745766


## Answer the following questions: 
- Which of the document stores performs better? Take into account the different metrics enumerated in the previous point. => elastic
- Which of the document stores is faster? => elastic
- Try to determine the other pros and cons of using sparse and dense document retrieval models.
